In [729]:
import os
import django
import pandas as pd
import numpy as np
import datetime
from django.db import transaction
from django.utils import timezone
from pathlib import Path
from django.db.models import Max

# os.environ.setdefault("DJANGO_SETTINGS_MODULE", "backend.settings")
# os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
# django.setup()

# from mainApp.models import Field, FieldByYear, Course, Semester, Faculty
import sqlite3

In [730]:
faculty_df = pd.read_csv("sylabus_data/faculty_data_2025_1_stopien.csv", sep=';', index_col=0)
course_df = pd.read_csv("sylabus_data/course_data_2025_1_stopien.csv", sep=';', index_col=0)
field_df = pd.read_csv("sylabus_data/field_data_2025_1_stopien.csv", sep=';', index_col=0)

In [731]:
faculty_df.head()

,name
0,Wydział Inżynierii Lądowej i Gospodarki Zasobami
1,Wydział Inżynierii Metali i Informatyki Przemy...
2,"Wydział Elektrotechniki, Automatyki, Informaty..."
3,"Wydział Informatyki, Elektroniki i Telekomunik..."
4,Wydział Inżynierii Mechanicznej i Robotyki


In [732]:
field_df.head()

,faculty_id,name,kind,type,description,specialization
0,0,Budownictwo,Studia inżynierskie I stopnia,Stacjonarne,Kierunek studiów Budownictwo należy do obszaru...,NaN
1,0,Budownictwo,Studia inżynierskie I stopnia,Niestacjonarne,Kierunek studiów Budownictwo należy do obszaru...,NaN
2,0,Inżynieria Górnicza,Studia inżynierskie I stopnia,Stacjonarne,Kierunek studiów Inżynieria Górnicza należy do...,NaN
3,0,Inżynieria Górnicza,Studia inżynierskie I stopnia,Niestacjonarne,Kierunek studiów Inżynieria Górnicza należy do...,NaN
4,0,Inżynieria i Zarządzanie Procesami Przemysłowymi,Studia inżynierskie I stopnia,Stacjonarne,Kierunek studiów Inżynieria i Zarządzanie Proc...,NaN


In [733]:
course_df.head()

,field_id,name,semester,ects,test_type,lecturer,description
0,0,Geologia,1,3.0,Zaliczenie,Katarzyna Cyran,Zajęcia prowadzone w trybie mieszanym. Deforma...
1,0,Geometria wykreślna,1,2.0,Zaliczenie,Krzysztof Pałac,Ćwiczenia polegają na samodzielnym wykonywaniu...
2,0,Grafika inżynierska i rysunek techniczny,1,3.0,Zaliczenie,Sebastian Olesiak,Rysunek techniczny z zastosowaniem tradycyjnyc...
3,0,Geodezja,1,3.0,Zaliczenie,Michał Strach,1. Podstawowe pojęcia geodezyjne. Organizacja ...
4,0,Fizyka I,1,3.0,Zaliczenie,Michał Ślęzak,Wykład: Głównym celem wykładów jest jakościowe...


### Mapowanie nazw wydziałów (id) na nowe (id) w tabeli field w przypadku różnic w bazie i pliku

In [734]:
cnx = sqlite3.connect('db.sqlite3')
sql_faculty_df = pd.read_sql_query("SELECT * FROM mainApp_faculty", cnx, index_col='id').iloc[:-2]

In [735]:
faculty_map = dict(zip(faculty_df.index.values, faculty_df.name))

faculty_df = pd.concat([sql_faculty_df, faculty_df])
faculty_df = faculty_df.drop_duplicates('name')
faculty_df.index = np.arange(1, len(faculty_df)+1)
faculty_df = faculty_df.fillna(1)
faculty_df['building_id'] = faculty_df['building_id'].astype('int16')
# faculty_df = faculty_df.reset_index()
faculty_df.head()

,name,building_id
1,Wydział Inżynierii Metali i Informatyki Przemy...,1
2,"Wydział Elektrotechniki, Automatyki, Informaty...",1
3,"Wydział Informatyki, Elektroniki i Telekomunik...",1
4,Wydział Inżynierii Mechanicznej i Robotyki,1
5,"Wydział Geologii, Geofizyki i Ochrony Środowiska",1


In [736]:
new_faculty_map = {}
for key, val in faculty_map.items():
    new_faculty_map[key] = faculty_df.index[faculty_df['name'] == val].tolist()[0]

field_df['faculty_id'] = field_df['faculty_id'].replace(new_faculty_map)
field_df['specialization'] = field_df['specialization'].fillna('')
field_df['formula'] = ''

field_level_map = {'Studia inżynierskie I stopnia': 'I stopień', 'Studia licencjackie I stopnia': 'I stopień'}
field_df = field_df.rename(columns={'kind': 'level'})
field_df['level'] = field_df['level'].replace(field_level_map)
field_df['type'] = field_df['type'].str.lower()

field_df = field_df.replace('Edukacja Techniczno - Informatyczna', 'Edukacja Techniczno-Informatyczna')

field_df.head()

,faculty_id,name,level,type,description,specialization,formula
0,16,Budownictwo,I stopień,stacjonarne,Kierunek studiów Budownictwo należy do obszaru...,,
1,16,Budownictwo,I stopień,niestacjonarne,Kierunek studiów Budownictwo należy do obszaru...,,
2,16,Inżynieria Górnicza,I stopień,stacjonarne,Kierunek studiów Inżynieria Górnicza należy do...,,
3,16,Inżynieria Górnicza,I stopień,niestacjonarne,Kierunek studiów Inżynieria Górnicza należy do...,,
4,16,Inżynieria i Zarządzanie Procesami Przemysłowymi,I stopień,stacjonarne,Kierunek studiów Inżynieria i Zarządzanie Proc...,,


### Mapowanie nazw kierunków (id) na nowe (id) w tabeli course w przypadku różnic w bazie i pliku

In [737]:
sql_field_df = pd.read_sql_query("SELECT * FROM mainApp_field", cnx, index_col='id')
sql_field_df = sql_field_df[sql_field_df['description'].notna()]    # usunięcie NULL
sql_field_df = sql_field_df.drop_duplicates(['name', 'type'])
sql_field_df[['description', 'specialization']] = sql_field_df[['description', 'specialization']].fillna('')
sql_field_df['faculty_id'] = ''
sql_field_df['level'] = ''
sql_field_df

,name,formula,description,specialization,type,faculty_id,level
id,,,,,,,
209,Edukacja Techniczno-Informatyczna,2*M+3*G1+G2,,,stacjonarne,,
210,Informatyka Stosowana,2*M+3*G1+G2,,,stacjonarne,,
211,Inżynieria Ciepła,2*M+3*G1+G2,,,stacjonarne,,
212,Inżynieria Materiałowa,2*M+3*G1+G2,,,stacjonarne,,
213,Inżynieria Obliczeniowa,2*M+3*G1+G2,,,stacjonarne,,
...,...,...,...,...,...,...,...
355,Informatyka Medyczna,2*M+3*G1+G2,,,stacjonarne,,
356,Komputerowo Wspomagana Inżynieria Materiałów B...,2*M+3*G1+G2,,,stacjonarne,,
357,Informatyka-Uczenie Maszynowe i Sztuczna Intel...,2*M+3*G1+G2,,,stacjonarne,,


In [738]:
field_df

,faculty_id,name,level,type,description,specialization,formula
0,16,Budownictwo,I stopień,stacjonarne,Kierunek studiów Budownictwo należy do obszaru...,,
1,16,Budownictwo,I stopień,niestacjonarne,Kierunek studiów Budownictwo należy do obszaru...,,
2,16,Inżynieria Górnicza,I stopień,stacjonarne,Kierunek studiów Inżynieria Górnicza należy do...,,
3,16,Inżynieria Górnicza,I stopień,niestacjonarne,Kierunek studiów Inżynieria Górnicza należy do...,,
4,16,Inżynieria i Zarządzanie Procesami Przemysłowymi,I stopień,stacjonarne,Kierunek studiów Inżynieria i Zarządzanie Proc...,,
...,...,...,...,...,...,...,...
87,15,Kulturoznawstwo,I stopień,stacjonarne,Kierunek Kulturoznawstwo łączy wiedzę humanist...,,
88,15,Socjologia,I stopień,stacjonarne,Celem oferty edukacyjnej proponowanej w ramach...,,
89,17,Informatyka - Zarządzanie Bezpieczeństwem Info...,I stopień,stacjonarne,Celem kształcenia na kierunku Informatyka – Za...,,
90,17,Informatyka (kierunek wspólny - WI),I stopień,stacjonarne,Kształcenie na studiach I stopnia realizuje ko...,,


In [739]:
# Zakładamy: sql_field_df oraz field_df są już wczytane

# 1. Przywróć kolumnę 'id' w sql_field_df (jeśli potrzebna później)
sql_field_df = sql_field_df.reset_index()

# 2. Merge tylko po wspólnych 'name' i 'type'
merged = pd.merge(
    sql_field_df,
    field_df[['name', 'type', 'description', 'specialization', 'faculty_id', 'level']],
    on=['name', 'type'],
    how='left',
    suffixes=('', '_new')
)

# 3. Uzupełnienie braków tylko jeśli oryginalna wartość to ''
for col in ['description', 'specialization', 'faculty_id', 'level']:
    merged[col] = merged[col].mask(merged[col] == '', merged[f'{col}_new'])

# 4. Usuń tymczasowe *_new kolumny
merged = merged.drop(columns=[f'{col}_new' for col in ['description', 'specialization', 'faculty_id', 'level']])

# 5. Przywróć 'id' jako indeks
merged = merged.set_index('id')

# 6. Dodaj nowe wiersze z field_df, których nie ma w sql_field_df
#    (czyli brakujące pary name+type)
existing_keys = set(zip(sql_field_df['name'], sql_field_df['type']))
new_rows = field_df[~field_df[['name', 'type']].apply(tuple, axis=1).isin(existing_keys)].copy()

# Dodaj nowe ID
last_id = sql_field_df['id'].max()
new_ids = range(last_id + 1, last_id + 1 + len(new_rows))
new_rows['id'] = new_ids

# Dodaj kolumny których nie ma w field_df (np. formula)
if 'formula' not in new_rows.columns:
    new_rows['formula'] = None

# Przywróć kolejność kolumn zgodną z merged
new_rows = new_rows[['id', 'name', 'formula', 'description', 'specialization', 'type', 'faculty_id', 'level']]
new_rows = new_rows.set_index('id')

# 7. Połącz wszystko razem
final_df = pd.concat([merged, new_rows])

In [740]:
final_df[final_df['name'].str.contains('-')]

,name,formula,description,specialization,type,faculty_id,level
id,,,,,,,
209,Edukacja Techniczno-Informatyczna,2*M+3*G1+G2,Program studiów I stopnia został ułożony w tak...,,stacjonarne,1.0,I stopień
285,Mikro- i Nanotechnologie w Biofizyce,2*M+3*G1+G2,Kierunek Mikro- i Nanotechnologie w biofizyce ...,,stacjonarne,13.0,I stopień
328,Informatyka-Data Science,2*M+3*G1+G2,NaN,NaN,stacjonarne,NaN,NaN
357,Informatyka-Uczenie Maszynowe i Sztuczna Intel...,2*M+3*G1+G2,NaN,NaN,stacjonarne,NaN,NaN
360,Informatyka (kierunek wspólny - WEAIiIB),,Kształcenie na studiach I stopnia realizuje ko...,,stacjonarne,2,I stopień
361,Technologie Przemysłu 4.0 (kierunek wspólny - ...,,Celem kształcenia na kierunku Technologie Prze...,,stacjonarne,2,I stopień
362,Nowoczesne Technologie w Kryminalistyce (kieru...,,Absolwent kierunku posiada wiedzę z zakresu: p...,,stacjonarne,3,I stopień
365,Technologia Chemiczna (kierunek wspólny - WIMiC),,Podjęcie studiów na kierunku Technologia chemi...,,stacjonarne,7,I stopień
369,Technologia Chemiczna (kierunek wspólny - WEiP),,Podjęcie studiów na kierunku Technologia chemi...,,stacjonarne,12,I stopień


In [741]:
final_df

,name,formula,description,specialization,type,faculty_id,level
id,,,,,,,
209,Edukacja Techniczno-Informatyczna,2*M+3*G1+G2,Program studiów I stopnia został ułożony w tak...,,stacjonarne,1.0,I stopień
210,Informatyka Stosowana,2*M+3*G1+G2,Studia prowadzone na kierunku Informatyka Stos...,,stacjonarne,13.0,I stopień
211,Inżynieria Ciepła,2*M+3*G1+G2,NaN,NaN,stacjonarne,NaN,NaN
212,Inżynieria Materiałowa,2*M+3*G1+G2,Celem kształcenia na kierunku Inżynieria Mater...,,stacjonarne,7.0,I stopień
213,Inżynieria Obliczeniowa,2*M+3*G1+G2,Celem kształcenia na kierunku „Inżynieria obli...,,stacjonarne,1.0,I stopień
...,...,...,...,...,...,...,...
368,Geotechnologie Otworowe,,Opis kierunku studiów Geotechnologie Otworowe ...,,niestacjonarne,10,I stopień
369,Technologia Chemiczna (kierunek wspólny - WEiP),,Podjęcie studiów na kierunku Technologia chemi...,,stacjonarne,12,I stopień
370,Informatyka - Zarządzanie Bezpieczeństwem Info...,,Celem kształcenia na kierunku Informatyka – Za...,,stacjonarne,17,I stopień


In [742]:
# Przywróć indeks jako kolumnę w field_df i final_df
field_df = field_df.reset_index()  # teraz 'id' z field_df to kolumna
final_df = final_df.reset_index()  # teraz 'id' z final_df to kolumna

# Utwórz klucz dopasowania: (name, type, level)
field_df['match_key'] = list(zip(field_df['name'], field_df['type'], field_df['level']))
final_df['match_key'] = list(zip(final_df['name'], final_df['type'], final_df['level']))

# Połącz stare ID z nowym ID przez wspólne klucze
merge_keys = pd.merge(
    field_df[['index', 'match_key']],      # 'index' = stare ID
    final_df[['id', 'match_key']],         # 'id' = nowe ID
    on='match_key',
    how='inner'
)

# Stwórz słownik: stare_id -> nowe_id
field_id_map = dict(zip(merge_keys['index'], merge_keys['id']))
field_id_map

{0: 317,
 1: 334,
 2: 319,
 3: 341,
 4: 320,
 5: 342,
 6: 321,
 7: 343,
 8: 326,
 9: 209,
 10: 267,
 11: 340,
 12: 306,
 13: 344,
 14: 213,
 15: 307,
 16: 346,
 17: 215,
 18: 268,
 19: 216,
 20: 336,
 21: 360,
 22: 289,
 23: 355,
 24: 218,
 25: 219,
 26: 361,
 27: 269,
 28: 318,
 29: 220,
 30: 221,
 31: 335,
 32: 362,
 33: 224,
 34: 270,
 35: 333,
 36: 363,
 37: 226,
 38: 227,
 39: 228,
 40: 323,
 41: 230,
 42: 347,
 43: 364,
 44: 233,
 45: 234,
 46: 232,
 47: 271,
 48: 272,
 49: 274,
 50: 273,
 51: 240,
 52: 337,
 53: 309,
 54: 276,
 55: 242,
 56: 212,
 57: 356,
 58: 365,
 59: 322,
 60: 277,
 61: 315,
 62: 278,
 63: 279,
 64: 366,
 65: 367,
 66: 368,
 67: 252,
 68: 345,
 69: 253,
 70: 339,
 71: 254,
 72: 348,
 73: 250,
 74: 349,
 75: 256,
 76: 283,
 77: 305,
 78: 369,
 79: 316,
 80: 258,
 81: 259,
 82: 210,
 83: 285,
 84: 325,
 85: 261,
 86: 262,
 87: 263,
 88: 264,
 89: 370,
 90: 371,
 91: 372}

In [743]:
# Podmień przestarzałe field_id na nowe
course_df['field_id'] = course_df['field_id'].map(field_id_map)

## Podsumowanie

In [744]:
faculty_df

,name,building_id
1,Wydział Inżynierii Metali i Informatyki Przemy...,1
2,"Wydział Elektrotechniki, Automatyki, Informaty...",1
3,"Wydział Informatyki, Elektroniki i Telekomunik...",1
4,Wydział Inżynierii Mechanicznej i Robotyki,1
5,"Wydział Geologii, Geofizyki i Ochrony Środowiska",1
6,Wydział Geodezji Górniczej i Inżynierii Środow...,1
7,Wydział Inżynierii Materiałowej i Ceramiki,1
8,Wydział Odlewnictwa,1
9,Wydział Metali Nieżelaznych,1
10,"Wydział Wiertnictwa, Nafty i Gazu",1


In [745]:
final_df

,id,name,formula,description,specialization,type,faculty_id,level,match_key
0,209,Edukacja Techniczno-Informatyczna,2*M+3*G1+G2,Program studiów I stopnia został ułożony w tak...,,stacjonarne,1.0,I stopień,"(Edukacja Techniczno-Informatyczna, stacjonarn..."
1,210,Informatyka Stosowana,2*M+3*G1+G2,Studia prowadzone na kierunku Informatyka Stos...,,stacjonarne,13.0,I stopień,"(Informatyka Stosowana, stacjonarne, I stopień)"
2,211,Inżynieria Ciepła,2*M+3*G1+G2,NaN,NaN,stacjonarne,NaN,NaN,"(Inżynieria Ciepła, stacjonarne, nan)"
3,212,Inżynieria Materiałowa,2*M+3*G1+G2,Celem kształcenia na kierunku Inżynieria Mater...,,stacjonarne,7.0,I stopień,"(Inżynieria Materiałowa, stacjonarne, I stopień)"
4,213,Inżynieria Obliczeniowa,2*M+3*G1+G2,Celem kształcenia na kierunku „Inżynieria obli...,,stacjonarne,1.0,I stopień,"(Inżynieria Obliczeniowa, stacjonarne, I stopień)"
...,...,...,...,...,...,...,...,...,...
144,368,Geotechnologie Otworowe,,Opis kierunku studiów Geotechnologie Otworowe ...,,niestacjonarne,10,I stopień,"(Geotechnologie Otworowe, niestacjonarne, I st..."
145,369,Technologia Chemiczna (kierunek wspólny - WEiP),,Podjęcie studiów na kierunku Technologia chemi...,,stacjonarne,12,I stopień,(Technologia Chemiczna (kierunek wspólny - WEi...
146,370,Informatyka - Zarządzanie Bezpieczeństwem Info...,,Celem kształcenia na kierunku Informatyka – Za...,,stacjonarne,17,I stopień,(Informatyka - Zarządzanie Bezpieczeństwem Inf...
147,371,Informatyka (kierunek wspólny - WI),,Kształcenie na studiach I stopnia realizuje ko...,,stacjonarne,17,I stopień,"(Informatyka (kierunek wspólny - WI), stacjona..."


In [746]:
course_df

,field_id,name,semester,ects,test_type,lecturer,description
0,317,Geologia,1,3.0,Zaliczenie,Katarzyna Cyran,Zajęcia prowadzone w trybie mieszanym. Deforma...
1,317,Geometria wykreślna,1,2.0,Zaliczenie,Krzysztof Pałac,Ćwiczenia polegają na samodzielnym wykonywaniu...
2,317,Grafika inżynierska i rysunek techniczny,1,3.0,Zaliczenie,Sebastian Olesiak,Rysunek techniczny z zastosowaniem tradycyjnyc...
3,317,Geodezja,1,3.0,Zaliczenie,Michał Strach,1. Podstawowe pojęcia geodezyjne. Organizacja ...
4,317,Fizyka I,1,3.0,Zaliczenie,Michał Ślęzak,Wykład: Głównym celem wykładów jest jakościowe...
...,...,...,...,...,...,...,...
7310,372,Theory of Fun in Game,6,4.0,Completing the classes,Jowita Guja,Theoretical aspects of pleasure-oriented game ...
7311,372,Pracownia problemowa - ścieżka graficzna (arty...,6,4.0,Zaliczenie,Alicja Panasiewicz,W ramach pracowni prolemowej artystycznej stud...
7312,372,Pracownia problemowa - ścieżka producencka,6,4.0,Zaliczenie,Jowita Guja,"Konsultacje, brainstorming, wybór tematów proj..."
7313,372,Pracownia problemowa - ścieżka programistyczna,6,4.0,Zaliczenie,Krzysztof Boryczko,W ramach pracowni problemowej student realizuj...


# Uzupełnienie braków i stworzenie tabel odwołań

In [747]:
final_df.loc[final_df['formula'] == '', 'formula'] = '2*M+3*G1+G2'
final_df = final_df.drop(columns='match_key')
final_df = final_df.fillna('')
final_df['faculty_id'] = final_df['faculty_id'].replace('', np.nan)
final_df['faculty_id'] = final_df['faculty_id'].astype('Int64').astype(str)
final_df['faculty_id'] = final_df['faculty_id'].replace('<NA>', '')

C:\Users\szyme\AppData\Local\Temp\ipykernel_8136\877014057.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  final_df['faculty_id'] = final_df['faculty_id'].replace('', np.nan)


In [748]:
final_df

,id,name,formula,description,specialization,type,faculty_id,level
0,209,Edukacja Techniczno-Informatyczna,2*M+3*G1+G2,Program studiów I stopnia został ułożony w tak...,,stacjonarne,1,I stopień
1,210,Informatyka Stosowana,2*M+3*G1+G2,Studia prowadzone na kierunku Informatyka Stos...,,stacjonarne,13,I stopień
2,211,Inżynieria Ciepła,2*M+3*G1+G2,,,stacjonarne,,
3,212,Inżynieria Materiałowa,2*M+3*G1+G2,Celem kształcenia na kierunku Inżynieria Mater...,,stacjonarne,7,I stopień
4,213,Inżynieria Obliczeniowa,2*M+3*G1+G2,Celem kształcenia na kierunku „Inżynieria obli...,,stacjonarne,1,I stopień
...,...,...,...,...,...,...,...,...
144,368,Geotechnologie Otworowe,2*M+3*G1+G2,Opis kierunku studiów Geotechnologie Otworowe ...,,niestacjonarne,10,I stopień
145,369,Technologia Chemiczna (kierunek wspólny - WEiP),2*M+3*G1+G2,Podjęcie studiów na kierunku Technologia chemi...,,stacjonarne,12,I stopień
146,370,Informatyka - Zarządzanie Bezpieczeństwem Info...,2*M+3*G1+G2,Celem kształcenia na kierunku Informatyka – Za...,,stacjonarne,17,I stopień
147,371,Informatyka (kierunek wspólny - WI),2*M+3*G1+G2,Kształcenie na studiach I stopnia realizuje ko...,,stacjonarne,17,I stopień


In [749]:
course_df['test_type'] = course_df['test_type'].str.lower()
course_test_type_map = {'completing the classes': 'zaliczenie', 'exam':'egzamin'}
course_df['test_type'] = course_df['test_type'].replace(course_test_type_map)
course_df = course_df.reset_index()

In [750]:
course_df

,index,field_id,name,semester,ects,test_type,lecturer,description
0,0,317,Geologia,1,3.0,zaliczenie,Katarzyna Cyran,Zajęcia prowadzone w trybie mieszanym. Deforma...
1,1,317,Geometria wykreślna,1,2.0,zaliczenie,Krzysztof Pałac,Ćwiczenia polegają na samodzielnym wykonywaniu...
2,2,317,Grafika inżynierska i rysunek techniczny,1,3.0,zaliczenie,Sebastian Olesiak,Rysunek techniczny z zastosowaniem tradycyjnyc...
3,3,317,Geodezja,1,3.0,zaliczenie,Michał Strach,1. Podstawowe pojęcia geodezyjne. Organizacja ...
4,4,317,Fizyka I,1,3.0,zaliczenie,Michał Ślęzak,Wykład: Głównym celem wykładów jest jakościowe...
...,...,...,...,...,...,...,...,...
7310,7310,372,Theory of Fun in Game,6,4.0,zaliczenie,Jowita Guja,Theoretical aspects of pleasure-oriented game ...
7311,7311,372,Pracownia problemowa - ścieżka graficzna (arty...,6,4.0,zaliczenie,Alicja Panasiewicz,W ramach pracowni prolemowej artystycznej stud...
7312,7312,372,Pracownia problemowa - ścieżka producencka,6,4.0,zaliczenie,Jowita Guja,"Konsultacje, brainstorming, wybór tematów proj..."
7313,7313,372,Pracownia problemowa - ścieżka programistyczna,6,4.0,zaliczenie,Krzysztof Boryczko,W ramach pracowni problemowej student realizuj...


In [751]:
# Przygotowanie danych
field_faculty_df = pd.DataFrame(final_df[['id', 'faculty_id']])
field_faculty_df = field_faculty_df.rename(columns={'id': 'field_id'})

sql_field_faculty_df = pd.read_sql_query("SELECT * FROM mainApp_field_faculty", cnx, index_col='id')
sql_field_faculty_df = sql_field_faculty_df.reset_index()

# Tworzenie mapy tylko dla wartości innych niż 29 i 30
field_faculty_map = dict(zip(sql_field_faculty_df['field_id'], sql_field_faculty_df['faculty_id']))
field_faculty_map = {k: v for k, v in field_faculty_map.items() if v not in [29, 30]}


field_faculty_df

,field_id,faculty_id
0,209,1
1,210,13
2,211,
3,212,7
4,213,1
...,...,...
144,368,10
145,369,12
146,370,17
147,371,17


In [752]:
course_field_df = pd.DataFrame(course_df[['index', 'field_id']])
course_field_df = course_field_df.rename(columns={'index': 'course_id'})

course_field_df

,course_id,field_id
0,0,317
1,1,317
2,2,317
3,3,317
4,4,317
...,...,...
7310,7310,372
7311,7311,372
7312,7312,372
7313,7313,372


# Wrzucenie do bazy danych

## faculty

In [753]:
faculty_df = faculty_df.reset_index()  # upewnij się, że 'index' jest kolumną
faculty_df = faculty_df.rename(columns={'index': 'id'})  # dopasuj do bazy

# Połączenie z bazą danych
conn = sqlite3.connect('db.sqlite3')
cursor = conn.cursor()

# Wstawianie danych — ignoruj, jeśli dany 'id' już istnieje
cursor.executemany(
    """
    INSERT OR IGNORE INTO mainApp_faculty (id, name, building_id)
    VALUES (?, ?, ?)
    """,
    faculty_df[['id', 'name', 'building_id']].values.tolist()
)

conn.commit()
conn.close()

## field

In [754]:
# Przygotuj dane
df_to_insert = final_df[['id', 'name', 'formula', 'description', 'specialization', 'type', 'level']]
df_to_insert = df_to_insert.reset_index(drop=True)

# Połączenie z bazą
conn = sqlite3.connect('db.sqlite3')
cursor = conn.cursor()

# 1. Wstaw nowe rekordy (ignoruj, jeśli id już istnieje)
cursor.executemany(
    """
    INSERT OR IGNORE INTO mainApp_field (id, name, formula, description, specialization, type, level)
    VALUES (?, ?, ?, ?, ?, ?, ?)
    """,
    df_to_insert.values.tolist()
)

# 2. Zaktualizuj description i specialization, jeśli są puste w istniejących rekordach
update_query = """
UPDATE mainApp_field
SET
    description = COALESCE(NULLIF(description, ''), ?),
    specialization = COALESCE(NULLIF(specialization, ''), ?),
    level = ?
WHERE id = ?
"""

# iteracja po wierszach final_df
for _, row in df_to_insert.iterrows():
    cursor.execute(update_query, (
        row['description'],
        row['specialization'],
        row['level'],
        row['id']
    ))

conn.commit()
conn.close()

OperationalError: table mainApp_field has no column named level

## Course

In [766]:
# conn = sqlite3.connect('db.sqlite3')
# cursor = conn.cursor()

# # Przykład: usuń rekordy, gdzie course_id lub field_id są podejrzanie duże
# cursor.execute("""
#     DELETE FROM mainApp_course
#     WHERE id >= 0
# """)

# conn.commit()
# conn.close()

In [767]:
df_to_insert = course_df.copy()
df_to_insert = df_to_insert.reset_index(drop=True)  # reset indeksu, usunięcie starego
df_to_insert['id'] = range(1, len(df_to_insert)+1)       # nowe id od 0

# Zmień nazwy kolumn zgodnie z tabelą w bazie danych
df_to_insert = df_to_insert.rename(columns={
    'ects': 'ECTS',
    'description': 'additional_info'
})

# Wybierz tylko kolumny potrzebne do wstawienia
df_to_insert = df_to_insert[['id', 'name', 'ECTS', 'test_type', 'lecturer', 'additional_info']]

# Połączenie z bazą danych
conn = sqlite3.connect('db.sqlite3')
cursor = conn.cursor()

# INSERT OR IGNORE — wstawia tylko jeśli dany id nie istnieje
cursor.executemany(
    """
    INSERT OR IGNORE INTO mainApp_course (id, name, ECTS, test_type, lecturer, additional_info)
    VALUES (?, ?, ?, ?, ?, ?)
    """,
    df_to_insert.values.tolist()
)

conn.commit()
conn.close()

## field_faculty

In [756]:
conn = sqlite3.connect('db.sqlite3')
cursor = conn.cursor()

# Iteracja po wierszach field_faculty_df
for _, row in field_faculty_df.iterrows():
    field_id = row['field_id']
    faculty_id = row['faculty_id']

    # Pomijaj, jeśli faculty_id jest pustym stringiem
    if faculty_id == '':
        continue

    # Sprawdź, czy istnieje rekord z danym field_id
    cursor.execute(
        "SELECT id, faculty_id FROM mainApp_field_faculty WHERE field_id = ?",
        (field_id,)
    )
    existing = cursor.fetchone()

    if existing is None:
        # Nie istnieje — wstaw nowy rekord
        cursor.execute(
            """
            INSERT INTO mainApp_field_faculty (field_id, faculty_id)
            VALUES (?, ?)
            """,
            (field_id, faculty_id)
        )
    else:
        # Istnieje — aktualizuj, jeśli faculty_id jest różne
        existing_faculty_id = existing[1]
        if str(existing_faculty_id) != str(faculty_id):
            cursor.execute(
                """
                UPDATE mainApp_field_faculty
                SET faculty_id = ?
                WHERE field_id = ?
                """,
                (faculty_id, field_id)
            )

conn.commit()
conn.close()

## course_field

In [771]:
course_field_df = course_field_df.copy().reset_index(drop=True)
course_field_df['id'] = range(1, len(course_field_df) + 1)
course_field_df['course_id'] = course_field_df['course_id']+1

In [ ]:
conn = sqlite3.connect('db.sqlite3')
cursor = conn.cursor()

# Wyczyść tabelę
cursor.execute("DELETE FROM mainApp_course_field")

# Wstaw dane z ręcznie ustawionym id
cursor.executemany(
    """
    INSERT INTO mainApp_course_field (id, course_id, field_id)
    VALUES (?, ?, ?)
    """,
    course_field_df[['id', 'course_id', 'field_id']].values.tolist()
)

conn.commit()
conn.close()